In [1]:
import ee
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [1]:
import ee 
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
# Load Sentinel-2 dataset
sentinel = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(ee.Geometry.Point([77.5946, 12.9716]))  # Example: Bangalore location

print(sentinel.size().getInfo())  # Print number of images in the collection


c:\Users\win10\OneDrive\Desktop\Final year Project\my_env\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


73


In [6]:
import ee
import folium
from folium import plugins

# Authenticate and initialize Earth Engine
ee.Initialize()
# Load Sentinel-2 dataset and filter by date and location
sentinel = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(ee.Geometry.Point([77.5946, 12.9716]))  # Example: Bangalore


In [7]:
# Visualization parameters
viz_params = {
    'bands': ['B4', 'B3', 'B2'],  # True color (RGB)
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# Reduce image collection to a single image (median composite)
sentinel_image = sentinel.median()


In [8]:
# Get URL for visualization
map_id_dict = ee.Image(sentinel_image).getMapId(viz_params)
tile_url = map_id_dict['tile_fetcher'].url_format


In [9]:
# Create a Folium map centered on Bangalore
map_center = [12.9716, 77.5946]  # Latitude, Longitude
m = folium.Map(location=map_center, zoom_start=10)

# Add Sentinel-2 layer to the map
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    overlay=True,
    name='Sentinel-2'
).add_to(m)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m.save("sentinel_map.html")
m


In [10]:
# Function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # NIR (B8) and Red (B4) bands
    return image.addBands(ndvi)

# Apply NDVI calculation to the Sentinel-2 collection
sentinel_ndvi = sentinel.map(calculate_ndvi)

# Reduce to a single image (median composite)
ndvi_image = sentinel_ndvi.median()

# Visualization parameters for NDVI
ndvi_viz_params = {
    'bands': ['NDVI'],
    'min': 0,
    'max': 1,
    'palette': ['blue', 'white', 'green']  # Color palette for NDVI
}

# Get the NDVI layer URL
ndvi_map_id_dict = ee.Image(ndvi_image).getMapId(ndvi_viz_params)
ndvi_tile_url = ndvi_map_id_dict['tile_fetcher'].url_format


In [11]:
# Add NDVI layer to the map
folium.TileLayer(
    tiles=ndvi_tile_url,
    attr='Google Earth Engine',
    overlay=True,
    name='NDVI'
).add_to(m)

# Add layer control to toggle between layers
folium.LayerControl().add_to(m)

# Save and render the map
m.save("ndvi_map.html")
m


In [ ]:
import ee

# Initialize the Earth Engine API
ee.Authenticate()  # Authenticate using the credentials in your browser
ee.Initialize()

# Define the area of interest (AOI) with specific coordinates
min_lon = -74.2591   # Example coordinates for New York City
min_lat = 40.4774
max_lon = -73.7004
max_lat = 40.9176

# Define the AOI geometry using the coordinates
aoi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Specify the satellite image collection (e.g., Landsat 8)
collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(aoi).filterDate('2019-01-01', '2020-12-31')

# Select bands (e.g., NIR, Red, Green for vegetation)
image = collection.select(['B5', 'B4', 'B3']).median().clip(aoi)

# Ensure the AOI is correctly handled and create a bounding box
region = aoi.bounds()

# Download the image as a GeoTIFF
download_url = image.getDownloadURL({
    'scale': 30,
    'region': region,  # Use bounds() to make sure it's correctly formatted
    'fileFormat': 'GeoTIFF'
})

print("Download link:", download_url)


EEException: Invalid format for "region" property. See Export.image() documentation for more details.